In [23]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Embedding, Conv1D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
airline_data = pd.read_csv('../data/Tweets.csv')
airline_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [25]:
airline_data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [26]:
X = airline_data['text']
y = pd.get_dummies(airline_data.airline_sentiment, prefix='sent').values
y.shape

(14640, 3)

In [27]:
def clean_text(doc):
    document = remove_tags(doc)
    
    document = re.sub('[^a-zA-Z]', ' ', document)
    
    document = re.sub(r'\s+[^a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\s+', ' ', document)
    
    return document

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(document):
    return TAG_RE.sub('', document)

In [28]:
X_sentences = []
reviews = list()

for rev in X:
    X_sentences.append(clean_text(rev))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_sentences, y, test_size=0.2, random_state=42)

In [30]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [31]:
vocabulary_size = len(tokenizer.word_index) + 1
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [32]:
embedded_dict = dict()
glove_embeddings = open('../data/glove.6B.100d.txt', encoding='utf-8')

In [33]:
for embeddings in glove_embeddings:
    embeddings_tokens = embeddings.split()
    emb_word = embeddings_tokens[0]
    emb_vector = np.asarray(embeddings_tokens[1:], dtype='float32')
    embedded_dict[emb_word] = emb_vector
glove_embeddings.close()

In [34]:
embedd_mat = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embedded_dict.get(word)
    if embedding_vector is not None:
        embedd_mat[index] = embedding_vector
embedd_mat.shape

(12085, 100)

In [35]:
def enbedded_model():
    embedding_inputs = Input(shape=(maxlen))
    embedding_layer = Embedding(vocabulary_size, 100, weights=[embedd_mat], trainable=False)(embedding_inputs)
    flatten_layer = Flatten()(embedding_layer)
    
    dense1 = Dense(512, activation='relu')(flatten_layer)
    do1 = Dropout(0.3)(dense1)

    dense2 = Dense(512, activation='relu')(do1)
    do2 = Dropout(0.3)(dense2)
    
    dense3 = Dense(512, activation='relu')(do2)
    do3 = Dropout(0.3)(dense3)
    
    output_layer = Dense(y_train.shape[1], activation='softmax')(do3)
    return Model(inputs=embedding_inputs, outputs=output_layer)

model = enbedded_model()

In [36]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 100)          1208500   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               5120512   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [38]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_split=0.2)
score = model.evaluate(X_test, y_test, verbose=0)
print(score[0])
print(score[1])

Epoch 1/100
147/147 [==============================] - 3s 17ms/step - loss: 0.8478 - acc: 0.6369 - val_loss: 0.7174 - val_acc: 0.7162
Epoch 2/100
147/147 [==============================] - 2s 15ms/step - loss: 0.6519 - acc: 0.7395 - val_loss: 0.6971 - val_acc: 0.7264
Epoch 3/100
147/147 [==============================] - 2s 15ms/step - loss: 0.5525 - acc: 0.7794 - val_loss: 0.6927 - val_acc: 0.7230
Epoch 4/100
147/147 [==============================] - 2s 16ms/step - loss: 0.4789 - acc: 0.8097 - val_loss: 0.7047 - val_acc: 0.7264
Epoch 5/100
147/147 [==============================] - 2s 17ms/step - loss: 0.3704 - acc: 0.8596 - val_loss: 0.7476 - val_acc: 0.7243
Epoch 6/100
147/147 [==============================] - 2s 17ms/step - loss: 0.3067 - acc: 0.8872 - val_loss: 0.9026 - val_acc: 0.7230
Epoch 7/100
147/147 [==============================] - 2s 16ms/step - loss: 0.2518 - acc: 0.9042 - val_loss: 0.9615 - val_acc: 0.7029
Epoch 8/100
147/147 [==============================] - 2s 17ms

Epoch 62/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0371 - acc: 0.9888 - val_loss: 1.7779 - val_acc: 0.7175
Epoch 63/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0337 - acc: 0.9905 - val_loss: 1.8478 - val_acc: 0.7187
Epoch 64/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0498 - acc: 0.9837 - val_loss: 1.8827 - val_acc: 0.7200
Epoch 65/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0339 - acc: 0.9890 - val_loss: 1.9697 - val_acc: 0.7017
Epoch 66/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0321 - acc: 0.9883 - val_loss: 2.0409 - val_acc: 0.7102
Epoch 67/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0412 - acc: 0.9855 - val_loss: 2.3099 - val_acc: 0.7149
Epoch 68/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0521 - acc: 0.9838 - val_loss: 1.7239 - val_acc: 0.7029
Epoch 69/100
147/147 [==============================] -